In [1]:
from Util.drissionpage_util import drissionPage

In [2]:
url = 'https://www.wjx.cn/vm/tL4FoDX.aspx'
drissionPage.driver.get(url)

True

In [3]:
from DrissionPage._functions.by import By
topic_element_list = drissionPage.get_elements(xpath_kind=By.CLASS_NAME, xpath='field ui-field-contain')
topic_element_list

[<ChromiumElement div class='field ui-field-contain' topic='1' id='div1' req='1' data-role='fieldcontain' type='4'>,
 <ChromiumElement div class='field ui-field-contain' topic='2' id='div2' req='1' data-role='fieldcontain' type='3'>,
 <ChromiumElement div class='field ui-field-contain' topic='3' id='div3' req='1' data-role='fieldcontain' type='3'>,
 <ChromiumElement div class='field ui-field-contain' topic='4' id='div4' req='1' data-role='fieldcontain' type='3'>,
 <ChromiumElement div class='field ui-field-contain' topic='5' id='div5' req='1' data-role='fieldcontain' type='4'>,
 <ChromiumElement div class='field ui-field-contain' topic='6' id='div6' req='1' data-role='fieldcontain' type='3'>,
 <ChromiumElement div class='field ui-field-contain' topic='7' id='div7' req='1' data-role='fieldcontain' type='3'>,
 <ChromiumElement div class='field ui-field-contain' topic='8' id='div8' req='1' data-role='fieldcontain' type='4'>,
 <ChromiumElement div class='field ui-field-contain' topic='9' i

In [7]:
import random
from pprint import pprint
topic_data_list = []
for topic in topic_element_list:
    topic_type = topic.attrs["type"]
    topic_answer_list = []
    topic_title = drissionPage.get_element_text(father_element=topic, xpath_kind=By.CLASS_NAME, xpath='topichtml')
    topic_answers = drissionPage.get_elements(father_element=topic, xpath_kind=By.XPATH, xpath='//div[@class="ui-controlgroup column1"]/div')
    # print(len(topic_answers))
    all_weight = 100
    true_count = 0
    for topic_answer in topic_answers:
        # print(topic_answer)
        # x = topic_answer.eles((By.XPATH,'./*'))
        # print(len(x))
        element = topic_answer.ele((By.CLASS_NAME,'label'))
        text = element.text
        answer = {
            "text": text,
            "weight": 0,
            "input": False,
        }
        if len(topic_answer.eles((By.XPATH,'./*'))) > 2:
            # print("填空题")
            weight = random.randint(1, 5)
            answer["weight"] = weight
            answer["input"] = True
            answer["input-text"] = ['无']
            all_weight -= weight
            true_count += 1
        topic_answer_list.append(answer)
    # print(topic_answer_list)
    aver_weight = int(round(all_weight / (len(topic_answers) - true_count), 2))
    for answer in topic_answer_list:
        if not answer["input"]:
            answer["weight"] = aver_weight
    # print()
    topic_data = {
        "topic-type": topic_type,
        "topic-title": topic_title,
        "topic-answer-list": topic_answer_list
    }
    topic_data_list.append(topic_data)
pprint(topic_data_list)

[{'topic-answer-list': [{'input': False, 'text': 'A. 法国电影、电视剧', 'weight': 25},
                        {'input': False, 'text': 'B. 法语书籍、报刊', 'weight': 25},
                        {'input': False, 'text': 'C. 社交媒体平台', 'weight': 25},
                        {'input': False,
                         'text': 'D. 学校法语或法国文化课程',
                         'weight': 25}],
  'topic-title': '您通过哪些途径获取法国文化相关信息?（可多选）【多选题】',
  'topic-type': '4'},
 {'topic-answer-list': [{'input': False, 'text': 'A. 经常参与', 'weight': 25},
                        {'input': False, 'text': 'B. 偶尔参与', 'weight': 25},
                        {'input': False, 'text': 'C. 听说过但未参与', 'weight': 25},
                        {'input': False, 'text': 'D. 从未听说过', 'weight': 25}],
  'topic-title': '您参与过中国面向法国举办的文化推广活动吗?',
  'topic-type': '3'},
 {'topic-answer-list': [{'input': False, 'text': 'A. 中医药文化', 'weight': 25},
                        {'input': False, 'text': 'B. 武术文化', 'weight': 25},
                        {'input': False, '

In [4]:
from Util.FileUtil import save_label_json
# 截取url最后的字符串
name = url.split('/')[-1].split('.')[0]
save_label_json(topic_data_list, name)

NameError: name 'topic_data_list' is not defined

In [8]:
import random
from Util.FileUtil import load_json, save_json
from Util.drissionpage_util import drissionPage
from selenium.webdriver.common.by import By
from time import sleep
import os
import random
def get_random_option_by_weight(topic_answer_list, random_count=1):
    """
    根据权重随机选择选项
    :param topic_answer_list: 选项列表
    :param random_count: 需要选择的数量,默认为1
    :return: 选中的选项索引列表（始终返回列表类型）
    """
    # 如果是多选题,随机生成选择数量
    if random_count > 1:
        random_count = random.randint(1, min(random_count, len(topic_answer_list)))

    # 获取所有选项的权重
    weights = [option["weight"] for option in topic_answer_list]

    # 根据权重随机选择指定数量的选项
    selected_options = random.choices(
        range(len(topic_answer_list)),
        weights=weights,
        k=random_count
    )

    # 去重并排序，始终返回列表
    return sorted(list(set(selected_options)))

def run(url, count=10):
    '''
    运行程序
    :param topic_data_list: 题目数据列表
    :param url: 问卷星地址
    :param count: 运行次数,默认为10
    '''
    drissionPage.driver.get(url)
    sleep(1)
    if drissionPage.get_element(xpath_kind=By.CLASS_NAME,xpath='layui-layer-btn1',timeout=2):
        drissionPage.click_element(xpath_kind=By.CLASS_NAME,xpath='layui-layer-btn1')
    name = url.split('/')[-1].split('.')[0]
    output_dir = os.path.join(os.getcwd(), "Output", f"data-{name}")
    answer_data_url = os.path.join(os.getcwd(), "Config", f"data-{name}.json")
    if not os.path.exists(answer_data_url):
        print(f"答案数据不存在,请先初始化程序并调整权重")
        return
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    topic_data_list = load_json(answer_data_url)
    topic_element_list = drissionPage.get_elements(xpath_kind=By.CLASS_NAME, xpath='field ui-field-contain')
    answer_records = []
    for index,topic_data in enumerate(topic_data_list):
        topic_answer_list = topic_data["topic-answer-list"]
        print(index, topic_data)
        answer_list = topic_element_list[index].eles((By.CLASS_NAME,'label'))
        print(f"当前题目:{topic_data['topic-title']}")
        record = {
            "题目序号": index + 1,
            "题目标题": topic_data['topic-title'],
            "选择答案": [],
            "填空答案": []
        }

        if topic_data["topic-type"] == "4":
            # 多选题
            random_index_list = get_random_option_by_weight(topic_answer_list, random_count=2)
            record["选择答案"] = random_index_list
            print(f"多选随机选择答案:{random_index_list}")
            for answer_index in random_index_list:
                answer_list[answer_index].click()
                if topic_answer_list[answer_index]["input"]:
                    input_text = topic_answer_list[answer_index]["input-text"]
                    record["填空答案"].extend(input_text)

        elif topic_data["topic-type"] == "3":
            # 单选题
            random_index = get_random_option_by_weight(topic_answer_list)[0]
            answer_list[random_index].click()
            record["选择答案"].append(random_index)
            print(f"单选随机选择答案:{random_index}")
            if topic_answer_list[random_index]["input"]:
                input_text = topic_answer_list[random_index]["input-text"]
                record["填空答案"].extend(input_text)

        answer_records.append(record)
    # break
    save_path = save_json(answer_records, os.path.join(output_dir, f"answer_records_{count}.json"))
    drissionPage.click_element(xpath_kind=By.ID, xpath='ctlNext')
    if drissionPage.get_element(xpath_kind=By.ID,xpath='rectMask',timeout=3):
        drissionPage.click_element(xpath_kind=By.ID, xpath='rectMask')
    if drissionPage.get_element(xpath_kind=By.ID,xpath='divdsc',timeout=3):
        print(f"记录答卷情况到路径:{save_path}")
        run(url, count-1)
    else:
        print(f"第{count}次运行失败,重新运行")
        run(url, count)
run(url,10)

Element not found: layui-layer-btn1
0 {'topic-type': '4', 'topic-title': '您通过哪些途径获取法国文化相关信息?（可多选）【多选题】', 'topic-answer-list': [{'text': 'A. 法国电影、电视剧', 'weight': 25, 'input': False}, {'text': 'B. 法语书籍、报刊', 'weight': 25, 'input': False}, {'text': 'C. 社交媒体平台', 'weight': 25, 'input': False}, {'text': 'D. 学校法语或法国文化课程', 'weight': 25, 'input': False}]}
当前题目:您通过哪些途径获取法国文化相关信息?（可多选）【多选题】
多选随机选择答案:[0, 1]
1 {'topic-type': '3', 'topic-title': '您参与过中国面向法国举办的文化推广活动吗?', 'topic-answer-list': [{'text': 'A. 经常参与', 'weight': 25, 'input': False}, {'text': 'B. 偶尔参与', 'weight': 25, 'input': False}, {'text': 'C. 听说过但未参与', 'weight': 25, 'input': False}, {'text': 'D. 从未听说过', 'weight': 25, 'input': False}]}
当前题目:您参与过中国面向法国举办的文化推广活动吗?
单选随机选择答案:2
2 {'topic-type': '3', 'topic-title': '在您看来，以下中国文化元素，法国民众对哪项最感兴趣?', 'topic-answer-list': [{'text': 'A. 中医药文化', 'weight': 25, 'input': False}, {'text': 'B. 武术文化', 'weight': 25, 'input': False}, {'text': 'C. 传统节日习俗', 'weight': 25, 'input': False}, {'text': 'D. 中国饮食文化', 'wei


KeyboardInterrupt



In [10]:
drissionPage.get_element(xpath_kind=By.ID,xpath='divdsc',timeout=3)

<ChromiumElement div class='description check__point-state' id='divdsc'>

In [43]:
def run(topic_data_list):
    topic_element_list = drissionPage.get_elements(xpath_kind=By.CLASS_NAME, xpath='field ui-field-contain')
    answer_records = []
    for index,topic_data in enumerate(topic_data_list):
        topic_answer_list = topic_data["topic-answer-list"]
        answer_list = topic_element_list[index].eles((By.CLASS_NAME,'label'))
        print(index,topic_data)
        record = {
            "题目序号": index + 1,
            "题目标题": topic_data['topic-title'],
            "选择答案": [],
            "填空答案": []
        }

        if topic_data["topic-type"] == "4":
            # 多选题
            random_index_list = get_random_option_by_weight(topic_answer_list, random_count=2)
            record["选择答案"] = random_index_list
            print(random_index_list)
            for answer_index in random_index_list:
                answer_list[answer_index].click()
                if topic_answer_list[answer_index]["input"]:
                    input_text = topic_answer_list[answer_index]["input-text"]
                    record["填空答案"].extend(input_text)

        elif topic_data["topic-type"] == "3":
            # 单选题
            random_index = get_random_option_by_weight(topic_answer_list)
            answer_list[random_index].click()
            record["选择答案"].append(random_index)
            if topic_answer_list[random_index]["input"]:
                input_text = topic_answer_list[random_index]["input-text"]
                record["填空答案"].extend(input_text)

        answer_records.append(record)
    print(answer_records)
run(topic_data_list)

0 {'topic-type': '4', 'topic-title': '您通过哪些途径获取法国文化相关信息?（可多选）【多选题】', 'topic-answer-list': [{'text': 'A. 法国电影、电视剧', 'weight': 25, 'input': False}, {'text': 'B. 法语书籍、报刊', 'weight': 25, 'input': False}, {'text': 'C. 社交媒体平台', 'weight': 25, 'input': False}, {'text': 'D. 学校法语或法国文化课程', 'weight': 25, 'input': False}]}
[0, 1, 2]
1 {'topic-type': '3', 'topic-title': '您参与过中国面向法国举办的文化推广活动吗?', 'topic-answer-list': [{'text': 'A. 经常参与', 'weight': 25, 'input': False}, {'text': 'B. 偶尔参与', 'weight': 25, 'input': False}, {'text': 'C. 听说过但未参与', 'weight': 25, 'input': False}, {'text': 'D. 从未听说过', 'weight': 25, 'input': False}]}
2 {'topic-type': '3', 'topic-title': '在您看来，以下中国文化元素，法国民众对哪项最感兴趣?', 'topic-answer-list': [{'text': 'A. 中医药文化', 'weight': 25, 'input': False}, {'text': 'B. 武术文化', 'weight': 25, 'input': False}, {'text': 'C. 传统节日习俗', 'weight': 25, 'input': False}, {'text': 'D. 中国饮食文化', 'weight': 25, 'input': False}]}
3 {'topic-type': '3', 'topic-title': '您参加中法文化交流活动的频率是?', 'topic-answer-list': [{'text'